# This notebook finds a circle (according to its colour) and publishes the radius and coordinates for its centre.

In [1]:
import sys
import colorsys
import json

import numpy
import cv2

import rospy

from std_msgs.msg import String
from sensor_msgs.msg import Image

# ROS uses a serial format to transmit images, therefore
# it's necessary to convert it to an array in order to 
# use the images with Numpy or OpenCV
from cv_bridge import CvBridge
cv_bridge = CvBridge()

In [2]:
# Don't forget to change according to the arm you are using...
limb = "right"

In [3]:
# Here we are using the option "disable_signals" because ROS and OpenCV
# try to control the same low level signals, so it's necessary to disable it
# on ROS to avoid conflicts.
rospy.init_node('image_reader', disable_signals=True, anonymous=True)

In [4]:
received_image = [None]

# This is the function rospy will call everytime an image is received
def callback_camera_image(data):
    try:
        # OpenCV uses "bgr8", but Numpy uses "rgb8".
        # You can use cv2.cvtColor(opencv_img, cv2.COLOR_BGR2RGB )
        # and convert from OpenCV style to Numpy.
        opencv_img = cv_bridge.imgmsg_to_cv2(data, "bgr8")
    except Exception as err:
        print err
    
    received_image[0] = opencv_img

In [5]:
# This is the function rospy will call on shutdown
def shutdown():
    cv2.destroyAllWindows()
    print "Finishing..."

In [6]:
rospy.on_shutdown(shutdown)

In [7]:
# Make sure the camera is available (open):
# rosrun baxter_tools camera_control.py -l
image_sub = rospy.Subscriber("/cameras/"+limb+"_hand_camera/image",Image,callback_camera_image)

In [8]:
# Publisher for the coordinates found by OpenCV
coord_pub = rospy.Publisher('/coordinates_from_opencv_'+limb,String, queue_size = 1)

In [9]:
window_name_orig = "BAXTER Camera Original"
window_name_cv2 = "BAXTER Camera CV2"

cv2.startWindowThread()
cv2.namedWindow(window_name_orig,cv2.WINDOW_NORMAL)
cv2.namedWindow(window_name_cv2,cv2.WINDOW_NORMAL)

In [10]:
rate = rospy.Rate(10)

In [ ]:
# http://docs.opencv.org/3.1.0/da/d22/tutorial_py_canny.html
# http://docs.opencv.org/2.4/modules/imgproc/doc/feature_detection.html?highlight=houghcircles#houghcircles
# http://www.pyimagesearch.com/2014/07/21/detecting-circles-images-using-opencv-hough-circles/


LowerHSV = (5, 50, 50)
UpperHSV = (30, 255, 255)


while True:
    original = received_image[0].copy()
    
    low_res = cv2.resize(original, (320,200))

    
    # Converts the image to gray scale
    gray = cv2.cvtColor(low_res, cv2.COLOR_BGR2GRAY)
    
    gray = cv2.equalizeHist(gray)
    
    # Converts the image to HSV
    hsv = cv2.cvtColor(low_res, cv2.COLOR_BGR2HSV)
    
    
    # Blurs the image
    # blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Apply a threshold binary filter
    # thresh = cv2.threshold(blurred, 60, 255, cv2.THRESH_BINARY)[1]
    
    # Finds edges
    # edges = cv2.Canny(gray,100,200)
    

    # Filters according to the colour
    mask = cv2.inRange(hsv, LowerHSV, UpperHSV)
    
    # The function names say everything...
    mask = cv2.erode(mask, None, iterations=2)
    mask = cv2.dilate(mask, None, iterations=2)

    
    #
    # Detecting circles in the image
    #
    
#     #
#     # Using cv2.HoughCircles
#     #
    
#     # Blurs the image
#     blurred = cv2.GaussianBlur(mask, (5, 5), 0)
    
#     circles = cv2.HoughCircles(blurred, cv2.cv.CV_HOUGH_GRADIENT, 1.2, 100)
    
#     # ensure at least some circles were found
#     if circles is not None:
#         # convert the (x, y) coordinates and radius of the circles to integers
#         circles = numpy.round(circles[0, :]).astype("int")

#         # loop over the (x, y) coordinates and radius of the circles
#         for (x, y, r) in circles:
#             # draw the circle in the output image, then draw a rectangle
#             # corresponding to the center of the circle
#             cv2.circle(low_res, (x, y), r, (0, 255, 0), 4)
#             cv2.rectangle(low_res, (x - 5, y - 5), (x + 5, y + 5), (0, 128, 255), -1)
            
    #
    # Using cv2.findContours
    #
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]
    minimum_radius = 5

    # only proceed if at least one contour was found
    if len(cnts) > 0:
        # find the largest contour in the mask, then use
        # it to compute the minimum enclosing circle and
        # centroid
        c = max(cnts, key=cv2.contourArea)
        ((x, y), radius) = cv2.minEnclosingCircle(c)
        M = cv2.moments(c)
        centre = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))

        # only proceed if the radius meets a minimum size
        if radius > minimum_radius:
            # draw the circle and centroid on the frame,
            # then update the list of tracked points
            cv2.circle(low_res, (int(x), int(y)), int(radius),(0, 255, 255), 2)
            cv2.circle(low_res, centre, 5, (0, 0, 255), -1)
            
            # Publishes the coordinates for the circle and its radius
            coord_pub.publish(String(json.dumps([centre[0],centre[1],radius])))
    
    cv2.imshow(window_name_orig, low_res)
    cv2.imshow(window_name_cv2, mask)
    rate.sleep()
    
    key = cv2.waitKey(1) & 0xFF
    # if the 'q' key is pressed, stop the loop
    if key == ord("q"):
        break

In [26]:
# Grab a value from the original window and convert it to HSV
# https://en.wikipedia.org/wiki/HSL_and_HSV
hsv_colorsys = colorsys.rgb_to_hsv(80, 50, 38)
hsv_colorsys[0]*180 # OpenCV hue values go from 0 to 180

30.0